In [2]:
import numpy as np
import os
import pandas as pd
import re
import requests
import kagglehub

C:\Users\andon\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Machine Learning Project

by Andon Gorchov (@thunderman913)

In [4]:
path = kagglehub.dataset_download("tsaustin/us-historical-stock-prices-with-earnings-data")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\andon\.cache\kagglehub\datasets\tsaustin\us-historical-stock-prices-with-earnings-data\versions\7


In [5]:
earnings_path = f"{path}\\stocks_latest\\earnings_latest.csv"
stock_prices_path = f"{path}\\stocks_latest\\stock_prices_latest.csv"

In [6]:
earnings_data = pd.read_csv(earnings_path)
stock_data = pd.read_csv(stock_prices_path)

In [8]:
stock_data_tidy = stock_data

stock_data_tidy.date = pd.to_datetime(stock_data_tidy.date)

In [9]:
stock_data_tidy = stock_data_tidy.rename(columns={'symbol': 'ticker'})

In [10]:
stock_data_tidy = stock_data_tidy.drop(columns=['split_coefficient'])

In [11]:
earnings_data_tidy = earnings_data
earnings_data_tidy.date = pd.to_datetime(earnings_data_tidy.date)

In [12]:
earnings_data_tidy = earnings_data_tidy.rename(columns={'symbol': 'ticker'})

In [13]:
earnings_data_tidy = earnings_data_tidy.drop(earnings_data_tidy[(earnings_data_tidy.eps.isna()) | (earnings_data_tidy.eps_est.isna())].index)
earnings_data_tidy = earnings_data_tidy.drop(columns=['qtr'])

In [14]:
earnings_data_tidy

,ticker,date,eps_est,eps,release_time
14,A,2012-11-19,0.800,0.84,post
15,A,2013-02-14,0.660,0.63,post
16,A,2013-05-14,0.670,0.77,post
17,A,2013-08-14,0.620,0.68,post
18,A,2013-11-14,0.760,0.81,post
...,...,...,...,...,...
168598,ZYXI,2020-02-27,0.077,0.09,post
168599,ZYXI,2020-04-28,0.063,0.09,post
168600,ZYXI,2020-07-28,0.086,0.09,post
168601,ZYXI,2020-10-27,0.053,0.04,post


Get the stock_data_tidy, which are in financial_indicators

In [15]:
stock_data_tidy = stock_data_tidy[stock_data_tidy['date'] > (earnings_data_tidy['date'].min() - pd.Timedelta(days=100))]

In [16]:
stock_data_tidy = stock_data_tidy[stock_data_tidy['date'] < (earnings_data_tidy['date'].max() + pd.Timedelta(days=5))]

In [51]:
# Sort the DataFrame by ticker and date
stock_data_tidy = stock_data_tidy.sort_values(by=['ticker', 'date'], ascending=[True, True])

# Function to get the price X days ago
def get_price_x_days_ago(df, days):
    return df['close'].shift(days)

# Function to get peak price and how many days ago it happened
def get_peak_price_and_days_ago(df):
    df['peak_price'] = df['close'].rolling(window=90, min_periods=1).max()
    peak_day_index = df['close'].rolling(window=90, min_periods=1).apply(lambda x: (len(x) - 1 - x.argmax()), raw=False)
    df['peak_days_ago'] = peak_day_index.fillna(0).astype(int)  # Fill NaN for rows that don't have enough data
    return df

# Apply the operations without grouping by ticker
def process_ticker_data(df):
    # Add columns for prices X days ago
    for days in [1,2,3,4,5, 10, 15, 30, 60, 90]:
        df[f'price_{days}_days_ago'] = df.groupby('ticker')['close'].shift(days)
        df[f'volume_{days}_days_ago'] = df.groupby('ticker')['volume'].shift(days)
    
    # Get peak price and peak days ago
    #df = df.groupby('ticker', group_keys=False).apply(get_peak_price_and_days_ago)
    return df

# Process the DataFrame without grouping
stock_data_tidy = process_ticker_data(stock_data_tidy)

In [53]:
stock_data_tidy = stock_data_tidy[stock_data_tidy.price_90_days_ago.isna() == False]

In [58]:
stock_data_tidy.to_pickle('stock_data_processed.pkl') # TODO if has pickle, don't perform above operations

In [61]:
stock_data_tidy

,ticker,date,open,high,low,close,close_adjusted,volume,peak_price,peak_days_ago,...,price_10_days_ago,volume_10_days_ago,price_15_days_ago,volume_15_days_ago,price_30_days_ago,volume_30_days_ago,price_60_days_ago,volume_60_days_ago,price_90_days_ago,volume_90_days_ago
8236388,A,2012-12-26,41.11,41.230,40.770,40.89,27.9994,2559600,41.39,5,...,39.70,4459100.0,38.13,2188400.0,36.82,2073400.0,38.580,3414700.0,40.48,2838300.0
8236495,A,2012-12-27,40.82,40.890,40.010,40.64,27.8967,2019800,41.39,6,...,40.51,6184500.0,38.66,3927500.0,36.33,3540600.0,38.450,4866500.0,37.15,12796100.0
8235777,A,2012-12-28,40.22,40.392,39.800,39.83,27.3407,2325100,41.39,7,...,39.80,3472700.0,38.32,3568700.0,36.14,3169600.0,38.860,4222900.0,37.22,5377700.0
8237686,A,2012-12-31,39.69,40.965,39.640,40.94,28.1026,3367600,41.39,8,...,39.85,2631900.0,37.81,3326400.0,35.75,5226100.0,38.870,3200300.0,36.68,3973000.0
8238640,A,2013-01-02,42.17,42.170,41.150,41.88,28.7479,6287700,41.88,0,...,40.74,3454600.0,38.56,3048800.0,35.81,6603100.0,38.980,2503700.0,37.13,5375900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24379782,ZYXI,2021-06-07,15.68,16.040,15.560,15.75,15.7500,200433,20.29,89,...,14.78,177150.0,13.87,225206.0,16.18,256053.0,16.980,518048.0,21.71,2041405.0
24379781,ZYXI,2021-06-08,15.84,16.200,15.700,16.10,16.1000,142973,19.99,83,...,14.69,239937.0,13.82,183651.0,16.36,254656.0,16.870,421452.0,20.29,1132735.0
24379780,ZYXI,2021-06-09,16.00,16.330,15.920,16.08,16.0800,168428,19.99,84,...,14.13,211229.0,14.41,232231.0,16.22,164233.0,16.330,463388.0,18.07,895844.0
24379778,ZYXI,2021-06-10,16.09,16.270,15.875,16.06,16.0600,130727,19.99,85,...,14.55,224490.0,14.25,165608.0,16.26,217496.0,16.075,483238.0,18.17,556870.0
